# CONCEPT 

- Simple하게 하나의 모델로 전체 학습하도록
- 변수를 제거해가며 진행

# STEP 01

In [1]:
import warnings
warnings.filterwarnings(action='ignore')

import os
import gc
import math
import random
import pickle
import pandas as pd
import numpy as np
import multiprocessing
from tqdm.auto import tqdm

from sklearn.metrics import f1_score

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset, sampler

from xgboost import XGBClassifier, XGBRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from imblearn.over_sampling import SMOTE, SMOTENC, SMOTEN

import shap
import catboost

In [2]:
device = torch.device('cuda:1') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda', index=1)

In [3]:
df_train = pd.read_csv('./data/train.csv').drop(columns=['id', 'father', 'mother', 'gender','trait'])
df_test = pd.read_csv('./data/test.csv').drop(columns=['id', 'father', 'mother', 'gender','trait'])              
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262 entries, 0 to 261
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   SNP_01  262 non-null    object
 1   SNP_02  262 non-null    object
 2   SNP_03  262 non-null    object
 3   SNP_04  262 non-null    object
 4   SNP_05  262 non-null    object
 5   SNP_06  262 non-null    object
 6   SNP_07  262 non-null    object
 7   SNP_08  262 non-null    object
 8   SNP_09  262 non-null    object
 9   SNP_10  262 non-null    object
 10  SNP_11  262 non-null    object
 11  SNP_12  262 non-null    object
 12  SNP_13  262 non-null    object
 13  SNP_14  262 non-null    object
 14  SNP_15  262 non-null    object
 15  class   262 non-null    object
dtypes: object(16)
memory usage: 32.9+ KB


In [4]:
# text 형태의 categorical 변수들을 숫자형태로 변경

for i in tqdm(range(1, 15+1)) :
    target = str(i) if i >= 10 else "0"+str(i)
    cols = sorted(df_train[f"SNP_{target}"].unique().tolist())
    df_train[f"SNP_{target}"] = df_train[f"SNP_{target}"].map(lambda x : 0 if x==cols[0] else (1 if x==cols[1] else 2))
    df_test[f"SNP_{target}"] = df_test[f"SNP_{target}"].map(lambda x : 0 if x==cols[0] else (1 if x==cols[1] else 2))

df_train.info(), df_test.info()

  0%|          | 0/15 [00:00<?, ?it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262 entries, 0 to 261
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   SNP_01  262 non-null    int64 
 1   SNP_02  262 non-null    int64 
 2   SNP_03  262 non-null    int64 
 3   SNP_04  262 non-null    int64 
 4   SNP_05  262 non-null    int64 
 5   SNP_06  262 non-null    int64 
 6   SNP_07  262 non-null    int64 
 7   SNP_08  262 non-null    int64 
 8   SNP_09  262 non-null    int64 
 9   SNP_10  262 non-null    int64 
 10  SNP_11  262 non-null    int64 
 11  SNP_12  262 non-null    int64 
 12  SNP_13  262 non-null    int64 
 13  SNP_14  262 non-null    int64 
 14  SNP_15  262 non-null    int64 
 15  class   262 non-null    object
dtypes: int64(15), object(1)
memory usage: 32.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   SNP_01  175 non

(None, None)

In [5]:
df_train['class_B'] = df_train['class'].map(lambda x : 1 if x=='B' else 0)
df_train['class_C'] = df_train['class'].map(lambda x : 1 if x=='C' else 0)

for i in range(1, 15+1) :
    target = str(i) if i >= 10 else "0"+str(i)
    target2 = df_train.groupby(f'SNP_{target}')['class_B', 'class_C'].sum()
    target2['total'] = target2[['class_B', 'class_C']].sum(axis=1)
    for j in range(len(target2)) :
        if (target2['total'][j] < 3) : # 전체 표본이 너무 적은 경우 제외(확률 반반으로)
            target2['class_B'][j] = 1
            target2['class_C'][j] = 1
            
    value = target2['class_B'] / (target2['class_B']+target2['class_C'])    

    df_train[f"SNP_{target}_ratio"] = df_train[f"SNP_{target}"].map(lambda x : value[0] if x==0 else (value[1] if x==1 else value[2]))
    df_test[f"SNP_{target}_ratio"] = df_test[f"SNP_{target}"].map(lambda x : value[0] if x==0 else (value[1] if x==1 else value[2]))    

df_train.drop(columns=['class_B', 'class_C'], inplace=True)
df_train.info(), df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262 entries, 0 to 261
Data columns (total 31 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   SNP_01        262 non-null    int64  
 1   SNP_02        262 non-null    int64  
 2   SNP_03        262 non-null    int64  
 3   SNP_04        262 non-null    int64  
 4   SNP_05        262 non-null    int64  
 5   SNP_06        262 non-null    int64  
 6   SNP_07        262 non-null    int64  
 7   SNP_08        262 non-null    int64  
 8   SNP_09        262 non-null    int64  
 9   SNP_10        262 non-null    int64  
 10  SNP_11        262 non-null    int64  
 11  SNP_12        262 non-null    int64  
 12  SNP_13        262 non-null    int64  
 13  SNP_14        262 non-null    int64  
 14  SNP_15        262 non-null    int64  
 15  class         262 non-null    object 
 16  SNP_01_ratio  262 non-null    float64
 17  SNP_02_ratio  262 non-null    float64
 18  SNP_03_ratio  262 non-null    

(None, None)

In [6]:
class Autoencoder(nn.Module):
    def __init__(self, encoding_dim):
        super().__init__()
        self.encoding_dim = encoding_dim
        self.encoder = nn.Sequential(
            nn.Linear(30, 32),
            nn.GELU(),
            nn.Linear(32, 8),
            nn.GELU(),
            nn.Linear(8, encoding_dim),
            nn.GELU(),
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 8),
            nn.GELU(),
            nn.Linear(8, 32),
            nn.GELU(),
            nn.Linear(32, 30)
        )

    def forward(self, x):
        x1 = self.encoder(x)
        x2 = self.decoder(x1)
        return x1, x2

def ae_train(model, data_loader, criterion, optimizer, device, epochs=10):
    model.to(device)
    for epoch in range(epochs):
        epoch_loss = 0
        for x in data_loader:
            x = x[0].to(device)
            _, x_hat = model(x)
            loss = criterion(x_hat, x)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f'Epoch {epoch}: loss = {epoch_loss / len(data_loader):.4f}')

In [7]:
class VAE(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(VAE, self).__init__()
        self.input_dim = input_dim
        self.latent_dim = latent_dim
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, input_dim*2),
            nn.ReLU(),
            nn.Linear(input_dim*2, input_dim),
            nn.ReLU(),
            nn.Linear(input_dim, input_dim//2),
            nn.ReLU()
            )
        self.fc_mu = nn.Linear(input_dim//2, latent_dim)
        self.fc_logvar = nn.Linear(input_dim//2, latent_dim)
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, input_dim//2),
            nn.ReLU(),
            nn.Linear(input_dim//2, input_dim),
            nn.ReLU(),
            nn.Linear(input_dim, input_dim*2),
            nn.ReLU(),
            nn.Linear(input_dim*2, input_dim)
        )

    def encode(self, x):
        h = self.encoder(x)
        return self.fc_mu(h), self.fc_logvar(h)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

def loss_fn(recon_x, x, mu, logvar):
    reconstruction_loss = F.mse_loss(recon_x, x, reduction='sum')
    kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return reconstruction_loss + kl_loss

def train(model, optimizer, train_loader, device):
    model.train()
    train_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_fn(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        
        optimizer.step()
        
    return train_loss / len(train_loader.dataset)

def test(model, test_loader, device):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for data in test_loader:
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_fn(recon_batch, data, mu, logvar).item()
    return test_loss / len(test_loader.dataset)

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X = torch.Tensor(df_train.drop(columns=['class']).to_numpy())

# Create a dataset and data loader
dataset = torch.utils.data.TensorDataset(X)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=True)

# Initialize the model, criterion, and optimizer
encoding_dim = 3
model = Autoencoder(encoding_dim)
criterion = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters())
epochs=10000

# Train the model
ae_train(model, data_loader, criterion, optimizer, device, epochs)

Epoch 0: loss = 0.8343
Epoch 1: loss = 0.5673
Epoch 2: loss = 0.3107
Epoch 3: loss = 0.2784
Epoch 4: loss = 0.2474
Epoch 5: loss = 0.2079
Epoch 6: loss = 0.1837
Epoch 7: loss = 0.1744
Epoch 8: loss = 0.1707
Epoch 9: loss = 0.1702
Epoch 10: loss = 0.1686
Epoch 11: loss = 0.1681
Epoch 12: loss = 0.1676
Epoch 13: loss = 0.1667
Epoch 14: loss = 0.1653
Epoch 15: loss = 0.1632
Epoch 16: loss = 0.1602
Epoch 17: loss = 0.1583
Epoch 18: loss = 0.1570
Epoch 19: loss = 0.1553
Epoch 20: loss = 0.1539
Epoch 21: loss = 0.1526
Epoch 22: loss = 0.1518
Epoch 23: loss = 0.1506
Epoch 24: loss = 0.1494
Epoch 25: loss = 0.1489
Epoch 26: loss = 0.1474
Epoch 27: loss = 0.1474
Epoch 28: loss = 0.1463
Epoch 29: loss = 0.1451
Epoch 30: loss = 0.1452
Epoch 31: loss = 0.1441
Epoch 32: loss = 0.1432
Epoch 33: loss = 0.1427
Epoch 34: loss = 0.1426
Epoch 35: loss = 0.1418
Epoch 36: loss = 0.1415
Epoch 37: loss = 0.1407
Epoch 38: loss = 0.1405
Epoch 39: loss = 0.1402
Epoch 40: loss = 0.1389
Epoch 41: loss = 0.1385
Ep

In [9]:
X1 = X.to(device)
X2 = torch.Tensor(df_test.to_numpy()).to(device)

pred_train = model(X1)
pred_test = model(X2)

# reconstruction error
loss_train = []
loss_test = []

for i in range(len(X1)) :
    loss = criterion(pred_train[1][i], X1[i]).detach().cpu().numpy()
    loss_train.append(loss)
for j in range(len(X2)) :
    loss = criterion(pred_test[1][j], X2[j]).detach().cpu().numpy()
    loss_test.append(loss)

# encoding values
enco_train = pred_train[0].detach().cpu().numpy()
enco_test = pred_test[0].detach().cpu().numpy()

trainLoss = pd.DataFrame(data=loss_train, columns=['ae_loss'])
testLoss = pd.DataFrame(data=loss_test, columns=['ae_loss'])

ae_train = pd.DataFrame(data=enco_train, columns=['ae_'+str(x) for x in range(enco_train.shape[1])])
ae_test = pd.DataFrame(data=enco_test, columns=['ae_'+str(x) for x in range(enco_train.shape[1])])

train2 = pd.concat([df_train, trainLoss, ae_train], axis=1)
test2 = pd.concat([df_test, testLoss, ae_test], axis=1)

train2.info(), test2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262 entries, 0 to 261
Data columns (total 35 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   SNP_01        262 non-null    int64  
 1   SNP_02        262 non-null    int64  
 2   SNP_03        262 non-null    int64  
 3   SNP_04        262 non-null    int64  
 4   SNP_05        262 non-null    int64  
 5   SNP_06        262 non-null    int64  
 6   SNP_07        262 non-null    int64  
 7   SNP_08        262 non-null    int64  
 8   SNP_09        262 non-null    int64  
 9   SNP_10        262 non-null    int64  
 10  SNP_11        262 non-null    int64  
 11  SNP_12        262 non-null    int64  
 12  SNP_13        262 non-null    int64  
 13  SNP_14        262 non-null    int64  
 14  SNP_15        262 non-null    int64  
 15  class         262 non-null    object 
 16  SNP_01_ratio  262 non-null    float64
 17  SNP_02_ratio  262 non-null    float64
 18  SNP_03_ratio  262 non-null    

(None, None)

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X = torch.Tensor(df_train.drop(columns=['class']).to_numpy())

input_dim = X.shape[1]
latent_dim = 3
batch_size = 64
num_epochs = 3000

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_data = X
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_data = X
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)

model = VAE(input_dim, latent_dim).to(device)
optimizer = torch.optim.Adam(model.parameters())

for epoch in range(num_epochs):
    train_loss = train(model, optimizer, train_loader, device)
    test_loss = test(model, test_loader, device)
    print(f"Epoch {epoch}: Train loss = {train_loss}, Test loss = {test_loss}")

Epoch 0: Train loss = 28.853770074043567, Test loss = 27.89867540898214
Epoch 1: Train loss = 27.50666226685502, Test loss = 26.67966734908009
Epoch 2: Train loss = 26.279489007614952, Test loss = 25.284075263802333
Epoch 3: Train loss = 24.798956485195013, Test loss = 23.67247754744901
Epoch 4: Train loss = 23.066523777619572, Test loss = 21.930787181126252
Epoch 5: Train loss = 21.12257985122331, Test loss = 19.34919668881948
Epoch 6: Train loss = 18.680343453210728, Test loss = 17.084957850798396
Epoch 7: Train loss = 16.118968293867038, Test loss = 14.35503591289957
Epoch 8: Train loss = 13.928755257875865, Test loss = 12.82807555453468
Epoch 9: Train loss = 12.523830908855409, Test loss = 11.892921695272431
Epoch 10: Train loss = 11.802282930330465, Test loss = 11.329262362181685
Epoch 11: Train loss = 10.7973337828658, Test loss = 10.598093324035178
Epoch 12: Train loss = 10.320420927673807, Test loss = 10.157007552285231
Epoch 13: Train loss = 10.095864070280818, Test loss = 10.

In [18]:
X1 = X.to(device)
X2 = torch.Tensor(df_test.to_numpy()).to(device)

pred_train = model(X1)
pred_test = model(X2)

# reconstruction error
loss_train = []
loss_test = []

for i in range(len(X1)) :
    loss = criterion(pred_train[0][i], X1[i]).detach().cpu().numpy()
    loss_train.append(loss)
for j in range(len(X2)) :
    loss = criterion(pred_test[0][j], X2[j]).detach().cpu().numpy()
    loss_test.append(loss)

# encoding values
enco_train = pred_train[1].detach().cpu().numpy()
enco_test = pred_test[1].detach().cpu().numpy()

trainLoss = pd.DataFrame(data=loss_train, columns=['vae_loss'])
testLoss = pd.DataFrame(data=loss_test, columns=['vae_loss'])

ae_train = pd.DataFrame(data=enco_train, columns=['vae_'+str(x) for x in range(enco_train.shape[1])])
ae_test = pd.DataFrame(data=enco_test, columns=['vae_'+str(x) for x in range(enco_train.shape[1])])

train3 = pd.concat([train2, trainLoss, ae_train], axis=1)
test3 = pd.concat([test2, testLoss, ae_test], axis=1)

train3.info(), test3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262 entries, 0 to 261
Data columns (total 39 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   SNP_01        262 non-null    int64  
 1   SNP_02        262 non-null    int64  
 2   SNP_03        262 non-null    int64  
 3   SNP_04        262 non-null    int64  
 4   SNP_05        262 non-null    int64  
 5   SNP_06        262 non-null    int64  
 6   SNP_07        262 non-null    int64  
 7   SNP_08        262 non-null    int64  
 8   SNP_09        262 non-null    int64  
 9   SNP_10        262 non-null    int64  
 10  SNP_11        262 non-null    int64  
 11  SNP_12        262 non-null    int64  
 12  SNP_13        262 non-null    int64  
 13  SNP_14        262 non-null    int64  
 14  SNP_15        262 non-null    int64  
 15  class         262 non-null    object 
 16  SNP_01_ratio  262 non-null    float64
 17  SNP_02_ratio  262 non-null    float64
 18  SNP_03_ratio  262 non-null    

(None, None)

In [19]:
train3.to_csv("train3.csv", index=False)
test3.to_csv("test3.csv", index=False)

In [193]:
train3

,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,...,SNP_14_ratio,SNP_15_ratio,ae_loss,ae_0,ae_1,ae_2,vae_loss,vae_0,vae_1,vae_2
0,2,1,0,1,1,0,0,2,0,2,...,0.655172,0.689320,0.051564,1.452074,-0.090390,0.445621,0.134736,0.797452,0.012432,0.016688
1,1,1,1,0,0,1,0,1,0,1,...,0.655172,0.689320,0.045184,0.896699,-0.071467,2.207833,0.165281,-0.118389,0.007749,0.011049
2,2,2,0,1,2,2,0,1,1,1,...,0.655172,0.689320,0.029422,4.283075,-0.055054,0.891292,0.207401,1.511401,-0.027977,0.016492
3,0,2,0,1,0,2,2,0,2,1,...,0.655172,0.272727,0.023120,6.807485,-0.098940,4.654676,0.275078,-1.365971,0.018216,0.007873
4,2,2,2,0,2,0,0,0,0,2,...,0.655172,0.506329,0.015139,2.463362,-0.097177,-0.145657,0.203913,1.246927,0.005438,-0.008809
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,1,1,0,1,2,1,0,1,0,2,...,0.655172,0.689320,0.027507,6.283892,-0.082028,1.372399,0.105361,1.177084,0.006013,0.003980
258,2,0,1,0,0,1,1,1,0,1,...,0.655172,0.506329,0.026623,1.136812,-0.124313,4.219052,0.097318,-0.098132,0.009544,0.017788
259,1,2,0,1,0,1,2,1,1,0,...,0.000000,0.272727,0.040151,5.948636,-0.050407,5.409632,0.125271,-1.092377,0.016834,0.005786
260,0,2,0,1,0,2,2,0,1,1,...,0.000000,0.272727,0.031537,6.605302,-0.099360,3.585393,0.060433,-1.179354,0.016730,0.008736


In [236]:
target_cols = train3.columns.tolist()[7:10]  + ['class']
check = train3.drop_duplicates(subset=target_cols)
check['class'].value_counts() / len(check) * 175

A    67.741935
B    56.451613
C    50.806452
Name: class, dtype: float64

In [237]:
check

,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,...,SNP_14_ratio,SNP_15_ratio,ae_loss,ae_0,ae_1,ae_2,vae_loss,vae_0,vae_1,vae_2
0,2,1,0,1,1,0,0,2,0,2,...,0.655172,0.689320,0.051564,1.452074,-0.090390,0.445621,0.134736,0.797452,0.012432,0.016688
1,1,1,1,0,0,1,0,1,0,1,...,0.655172,0.689320,0.045184,0.896699,-0.071467,2.207833,0.165281,-0.118389,0.007749,0.011049
2,2,2,0,1,2,2,0,1,1,1,...,0.655172,0.689320,0.029422,4.283075,-0.055054,0.891292,0.207401,1.511401,-0.027977,0.016492
3,0,2,0,1,0,2,2,0,2,1,...,0.655172,0.272727,0.023120,6.807485,-0.098940,4.654676,0.275078,-1.365971,0.018216,0.007873
4,2,2,2,0,2,0,0,0,0,2,...,0.655172,0.506329,0.015139,2.463362,-0.097177,-0.145657,0.203913,1.246927,0.005438,-0.008809
5,2,2,1,0,2,0,0,1,0,2,...,0.655172,0.689320,0.051853,2.077820,-0.090320,-0.047724,0.112114,1.382738,0.001330,-0.000033
9,2,2,1,0,2,1,0,2,0,1,...,0.655172,0.272727,0.008981,3.482652,-0.168571,5.169548,0.246958,0.864142,-0.003825,0.006722
10,2,1,1,1,1,1,0,1,1,2,...,0.655172,0.689320,0.050124,3.495900,-0.102480,1.059249,0.064102,0.713244,0.001102,0.017580
12,2,1,2,0,0,1,0,1,0,2,...,0.655172,0.272727,0.049896,0.906614,-0.155810,0.664510,0.162826,-0.009680,0.010924,0.026559
13,1,0,1,1,0,2,0,0,0,0,...,0.655172,0.689320,0.078351,1.049169,-0.070061,2.550609,0.211040,-0.365853,0.011251,-0.000889


In [206]:
target_cols.remove('class')
test3.drop_duplicates(subset=target_cols)

,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,...,SNP_14_ratio,SNP_15_ratio,ae_loss,ae_0,ae_1,ae_2,vae_loss,vae_0,vae_1,vae_2
0,1,2,0,1,0,1,2,1,1,1,...,0.000000,0.506329,0.166831,5.642057,-0.023668,3.152251,0.144563,-1.038103,0.013879,0.004878
1,2,1,2,2,2,0,0,0,0,2,...,0.655172,0.689320,0.216685,5.760644,-0.100615,1.679931,0.214011,1.359224,0.024534,0.000039
4,0,2,0,2,0,2,2,0,2,1,...,0.655172,0.272727,0.080097,7.721220,-0.093797,4.167721,0.096322,-1.497357,0.009801,0.009291
6,0,1,1,0,1,1,0,0,0,1,...,0.655172,0.506329,0.380005,2.020229,-0.046703,3.967607,0.137545,-0.479254,-0.003346,-0.002836
7,2,0,1,2,2,0,0,0,0,2,...,0.655172,0.689320,0.233976,3.305760,-0.087047,1.554819,0.236233,0.853061,0.009690,0.003606
11,2,2,0,2,2,1,1,1,0,2,...,0.655172,0.689320,0.172975,4.857063,-0.137661,1.077439,0.189684,1.536607,0.020073,0.007905
15,1,0,2,0,2,1,0,2,0,2,...,0.655172,0.689320,0.270085,1.916645,-0.137938,0.786880,0.200615,0.772530,-0.010120,0.027190
18,1,1,1,1,2,2,0,2,0,2,...,0.655172,0.506329,0.371069,2.073329,-0.139352,0.608346,0.164570,1.199935,0.010247,0.020097
134,0,0,1,1,2,0,0,2,0,2,...,0.655172,0.689320,0.577003,2.806244,-0.081024,0.228382,0.257584,0.547322,-0.005735,0.024441


In [183]:
# trait와 Label A의 상관계수가 1로, 다른 변수에 대한 영향력이 학습되지 않을 정도로 높으므로 배제해보고 진행
df_train2 = train3.copy()
df_test2 = test3.copy()

# G1 : A & notA / G2 : B & notB / G3 : C & notC => Regressor
X, y = df_train2.drop(columns=['class']), (df_train2['class'].values ) # original
X.iloc[:, :15] = X.iloc[:, :15].astype('category')
X_test = df_test2.copy()
X_test.iloc[:, :15] = X_test.iloc[:, :15].astype('category')

X1 = X.copy()
X2 = X.copy()
X3 = X.copy()

X_test1 = X_test.copy()
X_test2 = X_test.copy()
X_test3 = X_test.copy()

y1 = (y=='A').astype(int)
y2 = (y=='B').astype(int)
y3 = (y=='C').astype(int)

# strategy1 = {0:300, 1:300}
# smote1 = SMOTEN(sampling_strategy=strategy1, k_neighbors=50)
# strategy2 = {0:300, 1:300}
# smote2 = SMOTEN(sampling_strategy=strategy2, k_neighbors=50)
# strategy3 = {0:300, 1:600}
# smote3 = SMOTEN(sampling_strategy=strategy3, k_neighbors=50)

from imblearn.under_sampling import RandomUnderSampler

strategy1 = {0:10, 1:10}
strategy2 = {0:10, 1:10}
strategy3 = {0:10, 1:10}
smote1 = RandomUnderSampler(sampling_strategy=strategy1)
smote2 = RandomUnderSampler(sampling_strategy=strategy2)
smote3 = RandomUnderSampler(sampling_strategy=strategy3)

X11, y11 = smote1.fit_resample(X1, y1)
X22, y22 = smote2.fit_resample(X2, y2)
X33, y33 = smote3.fit_resample(X3, y3)

In [184]:
def catgbmc(inputX, inputY, validX, validY, params) :  
    var_categ = inputX.columns.tolist()[:15]
    model = CatBoostClassifier(
        cat_features=var_categ,
        **params,
        task_type='GPU',
        devices='0',
        )
  
    model.fit(
        inputX, inputY,
        eval_set=(inputX, inputY)
        );     

    pred = model.predict(validX)
    score = f1_score(validY, pred, average='macro')
    print(score)
    
    return model

In [185]:
def catgbmR(inputX, inputY, validX, validY, params) :  
    var_categ = inputX.columns.tolist()[:15]
    model = CatBoostRegressor(
        cat_features=var_categ,
        **params,
        task_type='GPU',
        devices='0',
        )
  
    model.fit(
        inputX, inputY,
        eval_set=(inputX, inputY)
        );     

    pred = model.predict(validX) > 0.5
    score = f1_score(validY, pred)
    print(score)
    
    return model

In [104]:
params = {'iterations':100,
          'learning_rate':0.1,
          # 'l2_leaf_reg' : 10,
          # 'grow_policy' : 'Lossguide',
          'auto_class_weights' : 'SqrtBalanced',
          'verbose':0,
          'random_seed':0}

model0 = catgbmc(X, y, X, y, params)

1.0


In [191]:
params = {'iterations':100,
          'learning_rate':0.3,
        #   'l2_leaf_reg' : 10,
        #   'grow_policy' : 'Depthwise',
          'verbose':0,
          'random_seed':0}

model1 = catgbmR(X11, y11, X1, y1, params)
model2 = catgbmR(X22, y22, X2, y2, params)
model3 = catgbmR(X33, y33, X3, y3, params)

0.9465648854961832
0.9083333333333333
0.7674418604651164


In [75]:
pd.DataFrame(data=model1.get_feature_importance(), index=model1.feature_names_, columns=['feature_importance']).sort_values('feature_importance', ascending=False)

,feature_importance
vae_0,95.688703
SNP_02,1.081806
ae_loss,0.716829
SNP_11,0.428488
SNP_12,0.378614
SNP_15,0.242201
SNP_13,0.185444
ae_1,0.149302
vae_loss,0.126855
SNP_01,0.124136


In [76]:
pd.DataFrame(data=model2.get_feature_importance(), index=model2.feature_names_, columns=['feature_importance']).sort_values('feature_importance', ascending=False)

,feature_importance
vae_0,91.081253
vae_2,2.858485
ae_1,1.889064
ae_2,0.601157
SNP_14,0.592063
SNP_12,0.447988
ae_loss,0.348154
SNP_09,0.216123
SNP_08,0.201903
SNP_02_ratio,0.176827


In [130]:
df = pd.read_csv("submit_high1.csv")
df['class'] = 'B'
df.to_csv("submit.csv", index=False)
df

,id,class
0,TEST_000,B
1,TEST_001,B
2,TEST_002,B
3,TEST_003,B
4,TEST_004,B
...,...,...
170,TEST_170,B
171,TEST_171,B
172,TEST_172,B
173,TEST_173,B


In [77]:
pd.DataFrame(data=model3.get_feature_importance(), index=model3.feature_names_, columns=['feature_importance']).sort_values('feature_importance', ascending=False)

,feature_importance
ae_0,42.157871
vae_0,25.615507
SNP_04_ratio,10.513975
SNP_04,2.833174
vae_2,2.478058
SNP_08,2.377650
vae_1,1.791499
SNP_03,1.308589
ae_loss,1.200529
ae_1,1.074583


In [107]:
pred1 = model1.predict(X_test1)
pred2 = model2.predict(X_test2)
pred3 = model3.predict(X_test3)

pred0 = model0.predict(X_test)

In [68]:
# pred3 = model1.predict_proba(df_test2.iloc[:, 1:])
# pred4 = model2.predict_proba(df_test2.iloc[:, 1:])

# pred5 = np.argmax((pred3+pred4) / 2, axis=1)
# pred5

In [108]:
df = pd.DataFrame()
df['pred_A'] = pd.Series(pred1.flatten())
df['pred_B'] = pd.Series(pred2.flatten())
df['pred_C'] = pd.Series(pred3.flatten())
df['high1'] = pd.read_csv('submit_high1.csv')['class']
df['high2'] = pd.read_csv('submit_high2.csv')['class']
df['pred_0'] = pd.Series(pred0.flatten())
df.head()

,pred_A,pred_B,pred_C,high1,high2,pred_0
0,0.961513,0.025238,0.100184,A,A,A
1,0.074034,1.030943,0.083862,B,B,B
2,0.100351,0.036408,1.016329,C,C,C
3,0.018103,0.727769,0.502003,C,C,B
4,0.965949,0.036810,0.050484,A,A,A


In [121]:
target = (model0.predict_proba(X_test) > 0.95).astype('int')

In [109]:
df.iloc[[3,5,12,119,126,162,168]]

,pred_A,pred_B,pred_C,high1,high2,pred_0
3,0.018103,0.727769,0.502003,C,C,B
5,0.027274,0.936984,0.469657,C,C,B
12,0.072923,0.690503,0.530483,B,C,B
119,0.090604,0.473318,0.634692,C,C,B
126,0.073298,0.747567,0.427800,B,C,C
162,0.012365,0.516311,0.557993,C,C,C
168,0.046510,0.953579,0.684606,B,B,B


In [122]:
np.sum(target)

86

In [123]:
target[[3,5,12,119,126,162,168]]

array([[0, 0, 0],
       [0, 1, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [82]:
df['new'] = 'D'
df.loc[df.pred_A > 0.9, 'new'] = 'A'
df.loc[df.pred_B > 0.9, 'new'] = 'B'
df.loc[df.pred_C > 0.5, 'new'] = 'C'

# df.loc[((df.pred_B > 0.5) & (df.pred_C < 0.4)), 'new'] = 'B'
df.new.value_counts()

B    78
A    50
C    42
D     5
Name: new, dtype: int64

In [83]:
df[df.new == 'D']

,pred_A,pred_B,pred_C,high1,high2,new
14,0.075850,0.804120,0.459858,B,B,D
60,0.155870,0.033594,0.113205,A,A,D
97,0.102144,0.830827,0.350064,B,B,D
122,0.008670,0.896284,0.161875,B,B,D
126,0.073298,0.747567,0.427800,B,C,D


In [ ]:
len(df[df.pred_A > 0.9]), len(df[df.pred_B > 0.8]), len(df[df.pred_C > 0.9])

In [ ]:
df[((df.pred_C > 0.1) & (df.pred_B < 0.9)) & ((df.high1 != 'C') | (df.high2 != 'C'))]

In [ ]:
df[df.pred1 != df.high1]

In [ ]:
df.pred1.value_counts()

In [ ]:
df.pred2.value_counts()

In [ ]:
df.high1.value_counts()

In [ ]:
df.high2.value_counts()

In [ ]:
df_train['class'].value_counts() / len(df_train) * len(df_test)

In [251]:
# A : 45, B : 78, C: 42
b_len = 83
answer = np.zeros(175)
answer[45:45+b_len] = 1
answer[45+b_len:] = 2

submit = np.ones(175)

# submit = answer.copy()
# submit[-1:]=0
# submit[45:45+1]=0

f1_score(answer, submit, average='macro'), f1_score(answer, submit, average='micro')

(0.2144702842377261, 0.4742857142857143)

In [248]:
answer

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2.])

In [246]:
# A : 45, B : 78, C: 42
b_len = 61
answer = np.ones(130)
answer[b_len:] = 0
# answer[45+b_len:] = 2

submit = np.ones(130)
# submit[b_len:] = 0

f1_score(answer, submit) / 3

0.21291448516579403